# House Prices Prediciton

## Imports

In [372]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.pipeline import Pipeline
from pyspark.ml.regression import LinearRegression


from sklearn import tree
from sklearn.tree import _tree
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

## Initialization

In [373]:
spark = SparkSession.builder.getOrCreate()

In [374]:
path = "./Master2/house-prices-prediction/train.csv"

train_df = spark.read.csv(path,header=True,inferSchema=True)

# Preprocessing : clean code 

## Filter null values

## Filter Numeric Features values

In [375]:
features_int =[]
for feature in train_df.dtypes:
    if feature[1]=='int':
        features_int.append(feature[0])

## Filter String Features Values

In [376]:
features_string =[]
for feature in train_df.dtypes:
    if feature[1]=='string':
        features_string.append(feature[0])

['MSZoning',
 'LotFrontage',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

## StringIndexer

In [377]:
indexers = [StringIndexer(inputCol = column, outputCol = column+"_index").fit(train_df) for column in features_string]
indexers_pipeline = Pipeline(stages = indexers)
indexers_df = indexers_pipeline.fit(train_df).transform(train_df)
# DROP STRING FEATURES
indexers_df = indexers_df.drop(*features_string)
# DROP ID COLUMN
indexers_df = indexers_df.drop("id")

indexers_df.toPandas()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,...,GarageYrBlt_index,GarageFinish_index,GarageQual_index,GarageCond_index,PavedDrive_index,PoolQC_index,Fence_index,MiscFeature_index,SaleType_index,SaleCondition_index
0,60,8450,7,5,2003,2003,706,0,150,856,...,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,20,9600,6,8,1976,1976,978,0,284,1262,...,9.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,60,11250,7,5,2001,2002,486,0,434,920,...,24.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,70,9550,7,5,1915,1970,216,0,540,756,...,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
4,60,14260,8,5,2000,2000,655,0,490,1145,...,11.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,7917,6,5,1999,2000,0,0,953,953,...,8.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,20,13175,6,6,1978,1988,790,163,589,1542,...,27.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1457,70,9042,7,9,1941,2006,275,0,877,1152,...,54.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
1458,20,9717,5,6,1950,1996,49,1029,0,1078,...,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Features and SalePrice Correlation

In [378]:
from os import truncate
from re import T
from pyspark.ml.linalg import DenseMatrix, Vectors
from pyspark.ml.stat import Correlation

corr_assem = VectorAssembler(inputCols=indexers_df.columns, outputCol='corr_features')
corr_vector = corr_assem.transform(indexers_df).select("corr_features")

pearsonCorr = Correlation.corr(corr_vector, 'corr_features', 'pearson').collect()[0][0]

corr_matrix = pearsonCorr.toArray().tolist()
corr_df = spark.createDataFrame(corr_matrix,indexers_df.columns)

correlation_colums = [
    "GrLivArea",
"TotalBsmtSF",
"1stFlrSF",
"TotRmsAbvGrd",
"FullBath",
"BsmtFinSF1",
"GarageCars", 
"2ndFlrSF",
"GarageArea",
"LotArea",
"YearBuilt",
"LotFrontage_index",
"YearRemodAdd",
"BsmtQual_index",
"OpenPorchSF",
"BsmtUnfSF", 
"MoSold",
"OverallCond", 
"WoodDeckSF", 
"KitchenQual_index",
"ExterQual_index",
"GarageType_index",
"MSSubClass",
"BsmtFullBath",
"MSZoning_index",
"ScreenPorch", 
"YrSold",
"BsmtExposure_index",
"GarageFinish_index",
"MasVnrArea_index"
]

# SELECT ONLY CORRELATION FEAUTRES
indexers_df = indexers_df.select(correlation_colums)

## Assembler

In [379]:
assem = VectorAssembler(inputCols=indexers_df.columns, outputCol='features')

## Pipeline

In [380]:

stages = [*indexers,assem]
pipeline = Pipeline(stages = stages)
final_data = pipeline.fit(train_df).transform(train_df)
final_data.select("features","SalePrice").show(5)

+--------------------+---------+
|            features|SalePrice|
+--------------------+---------+
|(80,[0,1,2,3,4,5,...|   208500|
|(80,[0,1,2,3,4,5,...|   181500|
|(80,[0,1,2,3,4,5,...|   223500|
|(80,[0,1,2,3,4,5,...|   140000|
|(80,[0,1,2,3,4,5,...|   250000|
+--------------------+---------+
only showing top 5 rows



# Machine learning

## Split Test and Train dataset

In [381]:
train, test = final_data.cache().randomSplit([0.8, 0.2], seed=42)

print("train count:" ,train.count())
print("test count:" ,test.count())



/usr/local/spark/python/pyspark/sql/pandas/conversion.py:186: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


train count: 1207


test count: 253


## Linear Regression

In [382]:
lr = LinearRegression(featuresCol="features",labelCol="SalePrice")

### Train section

In [383]:
train_model = lr.fit(train)
train_result = train_model.evaluate(train)

print("r2:",train_result.r2)
print("meanSquaredError: ",train_result.meanSquaredError)
print("meanAbsoluteError: ",train_result.meanAbsoluteError)

22/10/06 22:56:02 WARN Instrumentation: [01b3bf25] regParam is zero, which might cause numerical instability and overfitting.
22/10/06 22:56:03 WARN Instrumentation: [01b3bf25] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


r2: 0.9999999999690223
meanSquaredError:  0.19685729333818974
meanAbsoluteError:  0.3371560898118504


### Test section

In [384]:
unlabeled_data = test.select("features")
predictions = train_model.transform(unlabeled_data)
test.select("features","SalePrice").show(5)
predictions.show(5)

+--------------------+---------+
|            features|SalePrice|
+--------------------+---------+
|(80,[0,1,2,3,4,5,...|   223500|
|(80,[0,1,2,3,4,5,...|   307000|
|(80,[0,1,2,3,4,5,...|   129900|
|(80,[0,1,2,3,4,5,...|   279500|
|(80,[0,1,2,3,4,5,...|   139000|
+--------------------+---------+
only showing top 5 rows

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|(80,[0,1,2,3,4,5,...|223499.34373659428|
|(80,[0,1,2,3,4,5,...| 307000.0898447339|
|(80,[0,1,2,3,4,5,...|129899.72661419491|
|(80,[0,1,2,3,4,5,...| 279500.2528035681|
|(80,[0,1,2,3,4,5,...|138999.68319055278|
+--------------------+------------------+
only showing top 5 rows

